# 池化层

- 卷积对位置敏感
- 需要一定程度的平移不变性
  - 照明，物体位置，比例，外观等等因图像而异
- 二维最大池化
  - 返回滑动窗口中的最大值
- 返回滑动窗口中的最大值
  - 垂直边缘检测$\ a_{·,1}=1,a_{·,2}=1,a_{·,3}=0,a_{·,4}=0$
  - 卷积输出$\ a_{·,1}=0,a_{·,2}=1,a_{·,3}=0,a_{·,4}=0$
  - $2\times 2$最大池化$\ a_{·,1}=1,a_{·,2}=1,a_{·,3}=1,a_{·,4}=0$

## 填充，步幅和多个通道
- 池化层和卷积层类似，都有填充和步幅
- 没有学习的参数
- 在每个输入通道应用池化层以获得相应的输出通道
- 输出通道数=输入通道数

## 总结
- 池化层返回窗口中最大或平均值
- 缓解卷积层对位置的敏感性
- 同样有窗口大小、填充和步幅作为超参数

# 代码实现

池化层的正向传播

In [1]:
import torch
from torch import nn
import d2l_source.d2l as d2l

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i: i + p_h, j: j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i: i + p_h, j: j + p_w].mean()
    return Y

验证

In [2]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [3]:
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

填充和步幅

In [4]:
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

框架中的步幅和池化窗口大小相同

In [5]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

手动设置

In [6]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

任意大小的池化窗口

In [8]:
pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

池化层在每个输入通道上单独运算

In [9]:
X = torch.cat((X, X + 1), 1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

输出

In [10]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])